<a href="https://colab.research.google.com/github/JeevanChevula/Image-classification/blob/main/Image_Classification_(Cats_vs_Dogs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image
import os






Path to datasets

In [ ]:
train_dataset_path = "/content/drive/MyDrive/Colab Notebooks/Datasets/Image classification /training_set"
test_dataset_path = "/content/drive/MyDrive/Colab Notebooks/Datasets/Image classification /test_set"

In [ ]:
# Image dimensions and batch size
IMG_SIZE = 160
BATCH_SIZE = 16

Data Augmentation and Preprocessing for Training and Testing

In [ ]:

# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

#for test
test_datagen = ImageDataGenerator(rescale=1.0/255)

Loading Data

In [ ]:
# Load Training Data
train_generator = train_datagen.flow_from_directory(
    train_dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Load Test Data
test_generator = test_datagen.flow_from_directory(
    test_dataset_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Load and Fine-Tune MobileNetV2 with Custom Classifier

In [ ]:

# Load MobileNetV2  model
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# Custom Classifier with  3 Conv Layers
x = base_model.output
x = Conv2D(32, (3,3), activation='relu', padding='same')(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

# Create Model
model = Model(inputs=base_model.input, outputs=output)

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
EPOCHS = 5
training_progress = model.fit(train_generator, epochs=EPOCHS)

# Evaluate on Test Set
test_loss, test_acc = model.evaluate(test_generator)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 1367s 3s/step - accuracy: 0.9204 - loss: 0.2061
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 85s 170ms/step - accuracy: 0.9613 - loss: 0.1095
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 83s 166ms/step - accuracy: 0.9559 - loss: 0.1214
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 164ms/step - accuracy: 0.9622 - loss: 0.0989
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 164ms/step - accuracy: 0.9686 - loss: 0.0911
125/125 ━━━━━━━━━━━━━━━━━━━━ 907s 7s/step - accuracy: 0.9422 - loss: 0.1411


Saving results to a text file

In [ ]:
with open("results.txt", "w") as f:
    f.write("===== Model Training progress =====\n")
    f.write(f"Final Test Accuracy: {test_acc:.2%}\n")
    f.write(f"Final Test Loss: {test_loss:.4f}\n")
print(f"Final Test Accuracy: {test_acc:.2%}")
print(f"Final Test Loss: {test_loss:.4f}")
print("Results saved ")

Final Test Accuracy: 96.25%
Final Test Loss: 0.0905
Results saved 


In [ ]:
# Save Model
model.save("dogs_vs_cats_mobilenet.h5")

Testing model with an image

In [ ]:
def predict_single_image(img_path):
    model = load_model("dogs_vs_cats_mobilenet.h5")

    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    prediction = model.predict(img_array)

    if prediction[0][0] > 0.5:
        print(f"Predicted: Dog  ({prediction[0][0]:.2f})")
    else:
        print(f"Predicted: Cat  ({prediction[0][0]:.2f})")

predict_single_image("/content/drive/MyDrive/test image(dog).jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted: Dog  (0.99)
